In [1]:
from __future__ import division

import numpy as np
import pandas as pd
import cv2
from scipy.ndimage import zoom
import dlib
from tensorflow.keras.models import load_model
from imutils import face_utils
import pafy
import vlc
import time


In [2]:
global shape_x
global shape_y
global input_shape
global nClasses
global avg_list
global T
global S
global Q
global player

Emo_s={}
avg_list=[]
Angry=[]
Disgust=[]
Fear=[]
Happy=[]
Sad=[]
Surprise=[]
Neutral=[]
Emo_l=[]
player = None

In [3]:
Angry1 = pd.read_csv("./song/Angry.csv")
Anxiety1 = pd.read_csv("./song/Anxiety.csv")
Happy1 = pd.read_csv("./song/Happy.csv")
Neutral1 = pd.read_csv("./song/Neutral.csv")
Sad1 = pd.read_csv("./song/Sad.csv")

In [4]:
Sad1

,Emotion,Song,Time
0,Sad,https://youtu.be/o8uRydqx06g,334
1,Sad,https://youtu.be/wTynfy69U1U,258
2,Sad,https://youtu.be/xEObjH4fzao,229
3,Sad,https://youtu.be/dQ3QJoS5aaw,262
4,Sad,https://youtu.be/v0UvOsCi8mc,269
5,Sad,https://youtu.be/zYNsQ6ibdZs,350
6,Sad,https://youtu.be/nIVOFq3Xyzs,382
7,Sad,https://youtu.be/TgbdwLQDV3s,386
8,Sad,https://youtu.be/7ThDBzRLes8,271
9,Sad,https://youtu.be/MGYNJnXTlfc,294


In [5]:
Angry1 = pd.read_csv("./song/Angry.csv")
Anxiety1 = pd.read_csv("./song/Anxiety.csv")
Happy1 = pd.read_csv("./song/Happy.csv")
Neutral1 = pd.read_csv("./song/Neutral.csv")
Sad1 = pd.read_csv("./song/Sad.csv")
def show_webcam() :
    global Emo_s
    global Angry
    global Disgust
    global Fear
    global Happy
    global Sad
    global Surprise
    global Neutral
    global prediction
    Angry=[]
    Disgust=[]
    Fear=[]
    Happy=[]
    Sad=[]
    Surprise=[]
    Neutral=[]
    shape_x = 48
    shape_y = 48

    
    model = load_model('../Models/EmotionXCeption/video.h5')
    face_detect = dlib.get_frontal_face_detector()
    predictor_landmarks  = dlib.shape_predictor("../Models/Landmarks/face_landmarks.dat")
    
    video_capture = cv2.VideoCapture(0)
    T = time.time()
    print("\nDetect Face...")
    while True:
        ret, frame = video_capture.read()
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = face_detect(gray, 1)

        for (i, rect) in enumerate(rects):

            shape = predictor_landmarks(gray, rect)
            shape = face_utils.shape_to_np(shape)
            
            # divide by zero error handle
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            if x <=0 :
                x=1
            if y<= 0:
                y=1
            face = gray[y:y+h,x:x+w]
            
            face = zoom(face, (shape_x / face.shape[0],shape_y / face.shape[1]))
            
            face = face.astype(np.float32)
            
            face /= float(face.max())
            face = np.reshape(face.flatten(), (1, 48, 48, 1))
            
            prediction = model.predict(face)
            prediction_result = np.argmax(prediction)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            Angry.append(round(prediction[0][0],3))
            Disgust.append(round(prediction[0][1],3))
            Fear.append(round(prediction[0][2],3))
            Happy.append(round(prediction[0][3],3))
            Sad.append(round(prediction[0][4],3))
            Surprise.append(round(prediction[0][5],3))
            Neutral.append(round(prediction[0][6],3))

            if prediction_result == 0 :
                cv2.putText(frame, "Angry",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif prediction_result == 1 :
                cv2.putText(frame, "Disgust",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif prediction_result == 2 :
                cv2.putText(frame, "Fear",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif prediction_result == 3 :
                cv2.putText(frame, "Happy",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif prediction_result == 4 :
                cv2.putText(frame, "Sad",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif prediction_result == 5 :
                cv2.putText(frame, "Surprise",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else :
                cv2.putText(frame, "Neutral",(x+w-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            Emo_s={
                "Angry" : Angry,
                "Disgust" : Disgust,
                "Fear" : Fear,
                "Happy" : Happy,
                "Sad" : Sad,
                "Surprise" : Surprise,
                "Neutral" : Neutral
            }
            
        cv2.imshow('Video', frame)
        
        S = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q') or S-T>= 10:
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [6]:
def main():
    show_webcam()

In [7]:
def play_song(Emo):
    global player
    try:
        song_rd = Emo.sample()
        url  = song_rd.iloc[0][1]
        time_end = song_rd.iloc[0][2]
        print(url)
        video = pafy.new(url)
        best = video.getbest()
        playurl = best.url

        Instance = vlc.Instance("--no-xlib --quiet ")
        player = Instance.media_player_new()
        Media = Instance.media_new(playurl)
        Media.get_mrl()
        player.set_media(Media)
        player.play()
    except:
        return "ERROR : please check your camera or your internet"
    
    
    # Video time counter
    
    timeout = time.time() + time_end
    while True:
        if time.time() > timeout:
            player.stop()
            return "C"
            break
 

In [ ]:
if __name__ == "__main__":
    while True :
        main()
        Emo_s = pd.DataFrame(Emo_s)
        
        print(Emo_s.sum().idxmax())
        Emo = Emo_s.sum().idxmax() 
        print(Emo)
        if Emo == "Angry" :
            print(Angry1)
            V  = play_song(Angry1)
         
        if Emo == "Fear" :
            print(Anxiety1)
            V = play_song(Anxiety1)
        if Emo == "Disgust" :
            print(Anxiety1)
            V = play_song(Anxiety1)
        if Emo == "Happy"  :
            print(Happy1)
            V = play_song(Happy1)
        if Emo == "Surprise" :
            print(Happy1)
            V = play_song(Happy1)
        if Emo == "Sad" :
            print(Sad1)
            V = play_song(Sad1)
           
        if Emo == "Neutral" :
            print(Neutral1)
            V = play_song(Neutral1)
            


Detect Face...
Sad
Sad
   Emotion                          Song  Time
0      Sad  https://youtu.be/o8uRydqx06g   334
1      Sad  https://youtu.be/wTynfy69U1U   258
2      Sad  https://youtu.be/xEObjH4fzao   229
3      Sad  https://youtu.be/dQ3QJoS5aaw   262
4      Sad  https://youtu.be/v0UvOsCi8mc   269
5      Sad  https://youtu.be/zYNsQ6ibdZs   350
6      Sad  https://youtu.be/nIVOFq3Xyzs   382
7      Sad  https://youtu.be/TgbdwLQDV3s   386
8      Sad  https://youtu.be/7ThDBzRLes8   271
9      Sad  https://youtu.be/MGYNJnXTlfc   294
10     Sad  https://youtu.be/yz-Adq15Ouw   323
11     Sad  https://youtu.be/om-D4Y3ZUm0   350
https://youtu.be/o8uRydqx06g


In [ ]:
print(Emo )